# Master's thesis - Lukas Meuris - evaluation

This notebook contains the code to evaluate the models. 


In [ ]:
#!pip install git+https://github.com/google-research/weatherbench2.git

In [ ]:
import sys
sys.path.append("../")

import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs

import optax

import os
import time
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

import weatherbench2
from weatherbench2 import config

import matplotlib.pyplot as plt

# Specify input datasets

in this section we select the data need for the evaluation:
- obs_data: the ERA5 data, which will be used as a ground truth to compare to.
- pred_data: the forecast data to be evaluated. the WB2 evaluation API expects the dims as 'latitude' and 'longitude' so this needs to be changed back.
- climatology: used in some evaluations.

In [ ]:
# load obs data:
relative_path = "ERA5_data/1959-2022-6h-64x32_equiangular_conservative.zarr"
obs_path = os.path.join(os.path.dirname(os.getcwd()), relative_path)

In [ ]:
# load forecast data:
# """!change predictions path"""
# relative_path = "predictions/pred_64x32_2020_mae.zarr"
# pred_path = os.path.join(os.path.dirname(os.getcwd()), relative_path)
# pred_data = xr.open_zarr(pred_path)
# pred_data = pred_data.rename({'lon': 'longitude','lat': 'latitude'}) # rename the dimentions.
# pred_data

In [ ]:
"""!change predictions path"""
relative_path = "predictions/pred_64x32_2020_mae_eval.zarr"
pred_path = os.path.join(os.path.dirname(os.getcwd()), relative_path)
# pred_data.to_zarr(pred_path, mode='w') # changed data needs to be written to a different file to avoid corrupting the data.

In [ ]:
# load climatology:
relative_path = "ERA5_data/climatology.zarr"
climatology_path = os.path.join(os.path.dirname(os.getcwd()), relative_path)
climatology = xr.open_zarr(climatology_path)

# Set up WeatherBench configuration

Next, we will define a bunch of configuration instances to specify exactly what we want to evaluate.

## data configuration
The file paths are defined in a Paths config object, alongside an output directory:

In [ ]:
paths = config.Paths(
    forecast=pred_path,
    obs=obs_path,
    output_dir='../evaluation/',   # Directory to save evaluation results
)

In addition, we specify a Selection object that selects the variables and time period to be evaluated.

In [ ]:
selection = config.Selection(
    variables=[
        'geopotential',
        'temperature',
        'u_component_of_wind',
        'v_component_of_wind',
        '2m_temperature',
        '10m_u_component_of_wind',
        '10m_v_component_of_wind',
        'total_precipitation_6hr',
    ],
    levels=[500, 700, 850],
    time_slice=slice('2020-01-01', '2020-12-31'), #year 2020.
)

Together they make up the Data config:

In [ ]:
data_config = config.Data(selection=selection, paths=paths)

## selecting regions
we select the regions for which we want to do the evaluation.
here we split the regions in three separature groups (global, land1 and land2).
We do this to reduce the memory usage, because the evaluation is done completely in memory. 
                    

In [ ]:
from weatherbench2.regions import SliceRegion, LandRegion

land_sea_mask = xr.open_zarr(obs_path)['land_sea_mask'].compute()
# Default regions

global_region = {
      'global': SliceRegion(),
} 

land_regions1 = {
      'global_land': LandRegion(land_sea_mask=land_sea_mask),
      'europe': SliceRegion(
          lat_slice=slice(35, 75),
          lon_slice=[slice(360 - 12.5, None), slice(0, 42.5)],
      ),
      'north-america': SliceRegion(
          lat_slice=slice(25, 60), lon_slice=slice(360 - 120, 360 - 75)
      ),
}
land_regions2 = {
      'south-america': SliceRegion(
          lat_slice=slice(-55, 15), lon_slice=slice(285, 330)
      ),
      'asia': SliceRegion(
          lat_slice=slice(5, 60), lon_slice=slice(60, 150)
      ),
      'africa': SliceRegion(
          lat_slice=slice(-35, 37), lon_slice=slice(-20, 52.5)
      ),
      'ausnz': SliceRegion(
          lat_slice=slice(-45, -12.5), lon_slice=slice(120, 175)
      ),
  }

#### Evaluation configuration

Next, we can define which evaluation we want to run. To do so, we can define a dictionary of `config.Eval`s, each of which will be evaluated separately and saved to a different file. Eval instances contain the metrics objects, defined in metrics.py.

Note that for ACC, we additionally need to pass the climatology opened earlier.


In [ ]:
#define a wind vector error based on the U/V components of the wind.
from weatherbench2.metrics import WindVectorMSE, WindVectorRMSESqrtBeforeTimeAvg

def _wind_vector_error(err_type: str):
  """Defines Wind Vector [R]MSEs if U/V components are in variables."""
  if err_type == 'mse':
    cls = WindVectorMSE
  elif err_type == 'rmse':
    cls = WindVectorRMSESqrtBeforeTimeAvg
  else:
    raise ValueError(f'Unrecognized {err_type=}')
  wind_vector_error = []
  for u_name, v_name, vector_name in [
      ('u_component_of_wind', 'v_component_of_wind', 'wind_vector'),
      ('10m_u_component_of_wind', '10m_v_component_of_wind', '10m_wind_vector'),
  ]:
    wind_vector_error.append(
        cls(
            u_name=u_name,
            v_name=v_name,
            vector_name=vector_name,
        )
    )
  return wind_vector_error


In [ ]:
# define which metrics to evaluate.
from weatherbench2.metrics import MSE, ACC, Bias, MAE, SpatialBias, SpatialMSE, SpatialMAE, SEEPS, SpatialSEEPS

deterministic_metrics={
      'mse': MSE(wind_vector_mse=_wind_vector_error('mse')),
      'acc': ACC(climatology=climatology),
      'bias': Bias(),
}
spatial_metrics = {
      'bias': SpatialBias(),
}

deterministic_metrics['seeps_6hr'] = SEEPS(
    climatology=climatology,
    precip_name='total_precipitation_6hr',
    dry_threshold_mm=0.1,
)

In [ ]:
#define the evaluation configurations
"""!change save path"""
eval_configs = {
  'mae_det_global': config.Eval(
      metrics=deterministic_metrics,
      regions=global_region,
      evaluate_persistence=False),
   'mae_det_land1': config.Eval(
      metrics=deterministic_metrics,
      regions=land_regions1,
      evaluate_persistence=False),
    'mae_det_land2': config.Eval(
      metrics=deterministic_metrics,
      regions=land_regions2,
      evaluate_persistence=False),
   'mae_spatial': config.Eval(
      metrics=spatial_metrics,
      evaluate_persistence=False,
      output_format='zarr')
}

### Evaluate

Now, we can run the evaluation, this is done in memory, so may take up a lot of memory.

In [ ]:
from weatherbench2.evaluation import evaluate_in_memory

evaluate_in_memory(data_config, eval_configs)   # Takes some time.

Earlier we separated the regions to save on memory,
now they will be concatenated together again.

In [ ]:
# concat the deterministic evaluations:
det_global = xr.open_dataset('../evaluation/mae_det_global.nc')
det_land1 = xr.open_dataset('../evaluation/mae_det_land1.nc')
det_land2 = xr.open_dataset('../evaluation/mae_det_land2.nc')

det_global = xr.concat([det_global,det_land1], dim='region')
det_global = xr.concat([det_global,det_land2], dim='region')

"""!change save path"""
#write to file
relative_path = "evaluation/mae_det.nc"
pred_path = os.path.join(os.path.dirname(os.getcwd()), relative_path)
det_global.to_netcdf(pred_path, mode='w')
